# Installing Packages

In [ ]:
pip install langchain

In [ ]:
pip install langchain_google_genai

In [ ]:
pip install python-dotenv

# Importing Data

In [ ]:
import pandas as pd

In [ ]:
#Drive and sheets authentication
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
#Importing sheets data and converting and storing as pandas dataframe for operations
worksheet = gc.open('Sample Product Reviews').sheet1

# Get all values from the worksheet
rows = worksheet.get_all_values()

# Convert to a DataFrame, with the first row as the header
reviews_data = pd.DataFrame(rows[1:], columns=rows[0])

In [ ]:
reviews_data

# Gemini Setup

In [ ]:
import os
import getpass
from langchain_google_genai import ChatGoogleGenerativeAI

os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [ ]:
llm_model = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=os.environ["GOOGLE_API_KEY"],temperature=0.0)

# Propmting

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

## Output Parser Schema Design

review_language_schema = ResponseSchema(name="review_language",
                             description="Which language the review was written in?")
review_sentiment_schema = ResponseSchema(name="review_sentiment",
                                      description="What was the sentiment of the review")
translated_review_schema = ResponseSchema(name="translated_review",
                                    description="Reviews in other languages translated to English")
review_response_schema = ResponseSchema(name="review_response",
                                    description="Response to the review in the original language of the review")
translated_response_schema = ResponseSchema(name="translated_response",
                                    description="Response to the review translated to English.")


response_schemas = [review_language_schema,
                    review_sentiment_schema,
                    translated_review_schema,
                    review_response_schema,
                    translated_response_schema]

output_parser= StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

## Output Template
output_template= """
For the following text, extract the following information:

review_language: Identify the language in which the review is written like English, Spanish, French, etc.

review_sentiment: Tell me whether the customer review for the product was positive or negative. If unclear label it as Unidentified.

translated_review: If the review is in a language other than english give me the review translated in English. If it's already in english don't return anything.

review_response: Give me reply to the review in less than 30 words in the language of the original review. Keep a formal tone of response. Respond to both positive and negative reviews.
                Tailor responses accordingly to the sentiment of the review. I need a response for every review.

translated_response: Give me the above review_response translated into English. If it's already in english don't return anything.

Do not make stuff up.

Format the output as JSON with the following keys:
review_language
review_sentiment
translated_review
review_response
translated_response

Just give me the final json as output and nothing else.

Return Null if not appropriate answer available. Do not hallucinate.

text: {text}

{format_instructions}

"""

In [ ]:
print(output_template)

In [ ]:
chat=llm_model

# Generation

In [ ]:
for i in range(len(reviews_data)):

    prompt= ChatPromptTemplate.from_template(template=output_template)

    ## Making a prompt by looping over each item_description one by one
    messages= prompt.format_messages(text=reviews_data['Customer_Reviews'].iloc[i],
                                format_instructions=format_instructions)

    response= chat(messages)

    ## Getting the response and Parsing it to a json dictionary
    output_dict= output_parser.parse(response.content)

    ## Now store this dictionary output back to the Answer field
    reviews_data['Output'].iloc[i]= output_dict

    messages=""

In [ ]:
reviews_data

# Data Cleaning

In [ ]:
for col in ['Review_Language', 'Review_Sentiment', 'Translated_Review','Review_Response','Translated_Response']:
    if col not in reviews_data.columns:
        reviews_data[col] = None

In [ ]:
for i in range(len(reviews_data)):
  output=reviews_data['Output'].iloc[i]

  reviews_data['Review_Language'].iloc[i]= output.get('review_language')
  reviews_data['Review_Sentiment'].iloc[i]= output.get('review_sentiment')
  reviews_data['Translated_Review'].iloc[i]= output.get('translated_review')
  reviews_data['Review_Response'].iloc[i]= output.get('review_response')
  reviews_data['Translated_Response'].iloc[i]= output.get('translated_response')

In [ ]:
reviews_data

In [ ]:
# export reviews_data as a csv
reviews_data.to_csv('prepped_reviews_data.csv', index=False)